In [1]:
from strategy.weekly_price import WeeklyPrice
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
from backtester.backtester import Backtester as b
from database.market import Market
from preprocessor.model_preprocessor import ModelPreprocessor as mp
from modeler.modeler import Modeler as m
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import math

In [2]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,1).strftime("%Y-%m-%d")
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
strat = WeeklyPrice()
qfc = QuarterlyFinancialCategorical()

In [3]:
categories = qfc.retrieve_data("stock_category_sim")

In [4]:
categories.columns

Index(['_id', 'year', 'quarter', 'ticker', 'prediction'], dtype='object')

In [5]:
market.connect()
classification = market.retrieve_data("dataset_week_classification")
regression = market.retrieve_data("dataset_week_regression")
market.close()
classification = classification[:-2]
regression = regression[:-2]

In [6]:
gap = 5
week_gap = int(gap/5)
training_years = 1
quarterly_range = range(1,5)
yearly_range = range(2012,2021)

In [ ]:
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        quarter_categories = categories[(categories["year"] == year) & (categories["quarter"] == quarter)]
        for category in tqdm(quarter_categories["prediction"].unique()):
            try:
                group_tickers = quarter_categories[(quarter_categories["prediction"] == category)]
                prices = []
                if group_tickers.index.size > 0:
                    for ticker in group_tickers["ticker"].unique():
                        try:
                            ticker_class = classification.copy()[["year","quarter","week",ticker]].rename(columns={ticker:"y_class"})
                            ticker_regr = regression.copy()[["year","quarter","week",ticker]].rename(columns={ticker:str(0)})
                            test = ticker_regr.merge(ticker_class,on=["year","quarter","week"])
                            for i in range(1,14):
                                test[str(i)] = test[str(0)].shift(1)
                            test["y"] = test[str(0)].shift(-1)
                            test["y_class"] = test["y_class"].shift(-1)
                            f = test.dropna()
                            for col in ["year","quarter","week"]:
                                f[col] = [int(x) for x in f[col]]
                            f["date"] = [datetime(int(row[1]["year"]),int((row[1]["quarter"] * 3) - 2),1) for row in f.iterrows()]
                            f["ticker"] = ticker
                            prices.append(f)
                        except Exception as e:
                            print(str(e))
                            continue
                if len(prices) > 0:
                    p = pd.concat(prices)
                    training_data = p[(p["date"] < datetime(year,int((quarter*3)-2),1)) & (p["date"]>= datetime(year-training_years,int((quarter*3)-2),1))]
                    prediction_data = p[p["date"]==datetime(year,int((quarter*3)-2),1)]
                    X = training_data.drop(["quarter","year","y_class","y","date","ticker"],axis=1)
                    y = training_data[["y_class","y"]]
                    refined_data = {"X":X,"y":y}
                    regression_models = m.regression(refined_data.copy(),ranked=False,tf=False,xgb=True,sk=False,deep=False)
                    classification_models = m.classification(refined_data.copy(),tf=False,xgb=True,sk=False,deep=False,multioutput=False)
                    models = pd.DataFrame([regression_models,classification_models])
                    factors = refined_data["X"].columns
                    product = prediction_data[["year","quarter","week","ticker"]]
                    for i in range(models.index.size):
                        model = models.iloc[i]["model"]
                        model_type = models.iloc[i]["model_type"]
                        score = models.iloc[i]["score"]
                        predictions = model.predict(prediction_data[factors])
                        product[f"weekly_price_categorical_{model_type}_prediction"] = predictions
                        product[f"weekly_price_categorical_{model_type}_score"] = score
                    product["ticker"]
                    strat.db.connect()
                    strat.db.store_data("sim_categorical",product)
                    strat.db.close()
            except Exception as e:
                print(str(e))
                continue

In [5]:
strat.db.connect()
sim = strat.db.retrieve_data("sim_categorical")
strat.db.close()

In [6]:
market.connect()
price = market.retrieve_data("prices").drop("_id",axis=1)
market.close()

In [7]:
price["year"] = [x.year for x in price["date"]]
price["quarter"] = [x.quarter for x in price["date"]]

In [8]:
sim = sim.merge(price[["date","year","quarter","ticker","adjclose"]],on=["year","quarter","ticker"],how="left")
sim["weekly_predicted_delta"]  = (sim["weekly_price_categorical_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]

In [9]:
sim.drop("_id",axis=1,inplace=True)

In [10]:
# strat.db.connect()
# strat.db.store_data("weekly_sim",sim)
# strat.db.close()

In [11]:
datetime.now() - timedelta(days=datetime.now().weekday())

datetime.datetime(2021, 7, 26, 7, 55, 21, 386715)

In [12]:
# strat.db.connect()
# sim = strat.db.retrieve_data("weekly_sim")
# strat.db.close()

In [13]:
start = datetime(2013,1,7)
end = datetime(2020,12,23)
extreme = [True]
ceiling_settings = [True]
fixed = [True]
classification = [False]
weekly_value_settings = [True]
weekly_delta_range = range(0,12,3)
trade_signal_score_settings = range(81,104,5)
initial = 100

In [14]:
sim["weekly_price_categorical_regression_score"].mode()

0    0.996983
dtype: float64

In [15]:
sim = sim.groupby(["date","ticker"]).mean().reset_index()

In [16]:
sim = sim[sim["date"] >= start]

In [ ]:
epoch = 0
seats = 3
strat.db.connect()
strat.db.drop_table("m_epochs")
iterations = len(weekly_delta_range) * len(trade_signal_score_settings)
print(iterations)
for i in range(iterations):
    strat.db.drop_table("m_trades_{}".format(i))
for score in tqdm(trade_signal_score_settings):
    for delta in tqdm(weekly_delta_range):
        s = float(score/100)
        d = float(delta/100)
        epoch_dict = {"epoch":epoch
                      ,"score":s
                      ,"delta":d
                     }
        ts = b.backtest_weekly_adaptive(start,end,sim.copy(),seats,s,d)
        if len(ts) > 0:
            strat.db.store_data("m_epochs",pd.DataFrame([epoch_dict]))
            strat.db.store_data("m_trades_{}".format(epoch),pd.DataFrame(ts))
        epoch += 1
strat.db.close()

  0%|                                                                                                                                                               | 0/4 [00:00<?, ?it/s]

20
2018-12-31 00:00:00 stock had no more listed prices
